In [1]:
import sys, os 

import numpy as np
import pandas as pd

import scipy as sci
import xarray as xr

In [24]:
if False:
    # 1. download files from here: https://gitlab.hzdr.de/awi_paleodyn/growth_model_atlantic_cod_inputs/-/tree/main/soda
    input_files = xr.open_mfdataset('data/soda/*.nc') # soda_1958_2007
    new_time = pd.date_range('1958-01-01', '2008-01-01', freq='M')
    input_files = input_files.assign_coords({'time': new_time})
    input_files = input_files.sel(depth_coord=slice(30, 600),latitude = slice(51, 65), longitude = slice(-3,8))
    input_files.to_netcdf('data/SODA_North_Atlantic_1958_2007_sliced.nc')
else:
    input_files = xr.open_dataset('data/SODA_North_Atlantic_1958_2007_sliced.nc')

In [2]:
""" Definitions of growth model constants from Butzin and Pörtner, 2016  """

A_R = 8.660        # Rate of uninhibited growth at reference temperature T_R (% d^-1 g^1/b)
B_R = 0.3055       # Value of allometric exponent at reference temperature Tr
THETA_A = 18145    # Arrhenius temperature (K) for uninhibited reaction kinetics = 17871,85°C
THETA_B = 4258     # Arrhenius temperature (K) = 3984,85°C 
THETA_H =  25234   # Arrhenius temperature (K) for inhibited reaction kinetics = 24960,85°C
T_R = 283          # Reference optimum temperature (K) = 9.85°C
T_H = 286          # Temperature for inhibitive processes (K) = 12.85°C
C_AVG =  0.291     # Independent of temperature and weight constant (% d^-1)

In [22]:
Kelvin = 273.15       
def equation2(input_temp):
    temperature_kelvin = input_temp + Kelvin 
    a_numerator = A_R*np.exp(THETA_A/T_R - THETA_A/temperature_kelvin) 
    a_denominator = 1 + np.exp(THETA_H/T_H - THETA_H/temperature_kelvin)
    return a_numerator/a_denominator

def equation3(input_temp):
    """ Arrhenius equation """
    temperature_kelvin = input_temp + Kelvin
    return B_R*np.exp(THETA_B/T_R - THETA_B/temperature_kelvin)

In [19]:

# Specification of biological parameters  
# Number of years in one life cycle of an individual 
generation = 10
# The year when the input temeperature dataset starts 
# Should be one year less than starting year in the dataset
initial_year = 1958 # 2000    
# Number of years in the input temperature dataset
years = range(1958, 2008)  	# range(2000, 2005)  	 

# Define latitudes that you will use to save your weight-at-age data to netcdf
lat = np.array(input_files.latitude, dtype='f')
lon = np.array(input_files.longitude, dtype='f')
depths = np.array(input_files.depth_coord, dtype='f')

dt = 1

# Define dimensionality of coords
N_depths, N_lat, N_lon = len(depths), len(lat), len(lon)


In [23]:
results = {} # result by year
last_year = initial_year + generation  
while last_year < years[-1]:           
    age, weight = 0, np.ones(shape=(N_depths, N_lat*N_lon), dtype='f')
    growth_rates = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')
    age=0

    for every_year in range(initial_year, last_year):
        my_temp = input_files.thetao.sel(
            time=str(every_year), 
            depth_coord = depth_levels, 
            latitude = lat_coords, 
            longitude = lon_coords
        )            
        
        # Partly vectorize 4D temperature fields  to accelerate the computations
        temp_input_3d = my_temp.values.reshape(12, N_depths, N_lat*N_lon)             
            
        # Set NaN values
        temp_input_3d[np.where(temp_input_3d[:,:,:] <= -998)] = np.nan     

        # Initialize necessary variable fields
        a = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')
        b = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')

        # mm0 is never used
        if every_year == initial_year: mm0 = 2 
        else: mm0 = 0

        for mon in np.arange(0,12):
            for dd in np.arange(0,30):
                for ilev in np.arange(0, N_depths):  
                    a[ilev, :] = equation2(temp_input_3d[mon, ilev, :])
                    b[ilev, :] = equation3(temp_input_3d[mon, ilev, :]) * (-1.)
                    growth_rates[ilev, :] = 0.01 * ( a[ilev, :] * weight[ilev, :]** b[ilev, :] - C_AVG )  

                    growth_rates[ilev, :] = np.where(growth_rates[ilev, :] < 0,0, growth_rates[ilev, :])
                    weight[ilev, :] = weight[ilev, :] * (1. + dt * growth_rates[ilev, :])

        new_year = int(every_year) + 1
        
        # Reshape data to original shape
        a_3d = a.reshape(N_depths, N_lat, N_lon)
        b_3d = b.reshape(N_depths, N_lat, N_lon)

        growth_rates_3d = growth_rates.reshape(N_depths, N_lat, N_lon)

        # # 3D field with asymptotic weight
        weight_3d = 0.001 * weight.reshape((N_depths, N_lat, N_lon)) 
        
        # # Calculate maximum asymptotic weight at a given location 
        # # ("W*" in Butzin and Pörtner (2016)) 
        weight_max = np.nanmax(weight_3d, axis = 0)

        # optional
        content = {
            'age': age,
            'weight_3d': weight_3d,
            'weight_max': weight_max,
            'a_3d': a_3d,
            'B_3d': b_3d
        }
        if f'{every_year}' not in results.keys(): results[f'{every_year}'] = [content]
        else: results[f'{every_year}'].append(content)
        
        # Save netcdf
        # for i, (data, variable_name) in enumerate(zip(
        #     [a_3d, b_3d, growth_rates_3d, weight_3d, weight_max],
        #     ['a_3d', 'b_3d', 'growth_rates_3d', 'weight_3d', 'weight_max'],
        # )):
        #     save2netcdf(
        #         data=data, 
        #         var_name=variable_name, 
        #         directory=f'{out_dir}/init_initial_year_transient' , 
        #         year=every_year, 
        #         age=age, 
        #         dimension_labels=default_dim_labels if i < 4 else  ('latitude', 'longitude')
        #     )
        age += 1
            
    initial_year = initial_year + 1
    last_year = initial_year + generation 


<xarray.DataArray 'thetao' (time: 12, depth_coord: 18, latitude: 28, longitude: 22)>
dask.array<getitem, shape=(12, 18, 28, 22), dtype=float32, chunksize=(12, 18, 28, 22), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 1958-01-31 1958-02-28 ... 1958-12-31
  * longitude    (longitude) float64 -2.75 -2.25 -1.75 -1.25 ... 6.75 7.25 7.75
  * latitude     (latitude) float64 51.25 51.75 52.25 ... 63.75 64.25 64.75
  * depth_coord  (depth_coord) float32 30.0 40.0 50.0 60.0 ... 410.0 490.0 580.0
Attributes:
    long_name:   TEMPERATURE
    units:       Celsius_scale
    pointwidth:  1.0
    fnname:      flaggt


/var/folders/j1/k2clnjz530q64v44gw4srhpw0000gn/T/ipykernel_32035/1685685677.py:54: RuntimeWarning: All-NaN slice encountered
  weight_max = np.nanmax(weight_3d, axis = 0)


<xarray.DataArray 'thetao' (time: 12, depth_coord: 18, latitude: 28, longitude: 22)>
dask.array<getitem, shape=(12, 18, 28, 22), dtype=float32, chunksize=(12, 18, 28, 22), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 1959-01-31 1959-02-28 ... 1959-12-31
  * longitude    (longitude) float64 -2.75 -2.25 -1.75 -1.25 ... 6.75 7.25 7.75
  * latitude     (latitude) float64 51.25 51.75 52.25 ... 63.75 64.25 64.75
  * depth_coord  (depth_coord) float32 30.0 40.0 50.0 60.0 ... 410.0 490.0 580.0
Attributes:
    long_name:   TEMPERATURE
    units:       Celsius_scale
    pointwidth:  1.0
    fnname:      flaggt
<xarray.DataArray 'thetao' (time: 12, depth_coord: 18, latitude: 28, longitude: 22)>
dask.array<getitem, shape=(12, 18, 28, 22), dtype=float32, chunksize=(12, 18, 28, 22), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 1960-01-31 1960-02-29 ... 1960-12-31
  * longitude    (longitude) float64 -2.75 -2.25 -1.75 -1.25 ... 6.75 7.25

KeyboardInterrupt: 